In [7]:
import pandas as pd
import plotly as pt
import plotly.express as px
import plotly.graph_objs as go
import re
import numpy as np

In [10]:
## HNS CLEANING ##

name = "/Users/sabrinalong/Downloads/3-all.csv"
exportname = "a"
hns3all = pd.read_csv(name)

hns3all.columns = ['type', 'fy2020', 'fy2019', 'curryear2prior', 'fy2020budget', 'na', 'curryear2priorbudget']
hns3all.drop('na', axis=1, inplace=True)
hns3all.drop([0], inplace=True)
hns3all.reset_index(inplace=True)
hns3all.drop('index', axis=1, inplace=True)

col = []
for label in ['Hartford', 'New Haven', 'Stamford', 'System']:
    col.append([label] * 11)
newcol = sum(col, [])
newcol = pd.Series(newcol)

hns3all.drop([0, 12, 24, 36], inplace=True)
hns3all.reset_index(drop=True, inplace=True)
hns3all = pd.concat([hns3all, newcol], axis=1)
hns3all.rename(columns={
        0: 'line'
    }, inplace=True)
line = hns3all['line']
hns3all.drop(labels=['line'], axis=1, inplace=True)
hns3all.insert(0, 'line', line)

needs_fixing = ['fy2020', 'fy2019', 'curryear2prior', 'fy2020budget', 'curryear2priorbudget']

dfdict = {}
ls1 = []
ls2 = []
new_col_names = []

for column in needs_fixing:
    new1 = []
    new2 = []
    col = hns3all[column]
    
    for pair in col:
        split = pair.split(' ')
        res1 = re.sub('[^0-9]','', split[0])
        res2 = re.sub('[^0-9]','', split[1])
        
        new1.append(int(res1))
        new2.append(int(res2)/100)
        
    newname1 = column + "count"
    newname2 = column + "percentchange"  
    new_col_names.append(newname1)
    new_col_names.append(newname2)
    dfdict[newname1] = new1
    dfdict[newname2] = new2

sep_columns = pd.DataFrame(dfdict, columns = new_col_names)
hns_clean = hns3all[['line', 'type']]
final = pd.concat([hns_clean, sep_columns], axis=1)
final.fillna("Line Total", inplace=True)
final.to_csv(exportname)

In [4]:
# CT Covid Cleaning ##

name = "data/CTCOVID/Tests, Cases, Deaths by Town 7-13.csv"
exportname = _
ctcovid = pd.read_csv(name)

covid_dates = pd.to_datetime(ctcovid.last_update_date, format='%m/%d/%Y', errors='coerce')
ctcovid['date'] = covid_dates

ctcovid.rename(columns={
            'Total cases ': 'total_cases'
    }, inplace=True)

all_data = [ctcovid]
for df in all_data:
    df.columns = [col.lower() for col in df]
    df.columns = [col.replace(" ", "_") for col in df]
    
df.drop('Unnamed: 0', axis=1, inplace=True)

df.to_csv(exportname)

In [26]:
# BUS STOP WEEKLY RIDERS 

names_to_find =[]
data_sheets = []
all_dates = np.array([['03/01 - 03/07', '03/08 - 03/14', '03/15 - 03/21', '03/22 - 03/28',
       '03/29 - 04/04', '04/05 - 04/11', '04/12 - 04/18', '04/19 - 04/25',
       '04/26 - 05/02', '05/03 - 05/09', '05/10 - 05/16', '05/17 - 05/23',
       '05/24 - 05/30', '05/31 - 06/06', '06/07 - 06/13', '06/14 - 06/20',
       '06/21 - 06/27', '06/28 - 07/04', '07/05 - 07/11', '07/12 - 07/18']])
all_dates = np.swapaxes(all_dates, 0, 1)

name = "data/Bus Ridership/xlsx/Stamford Stops Wkly Total Lat-Lon.xlsx"
exportname = "data/Bus Ridership/csv/Stamford Weekly Bus Ridership.csv"
xlsx = pd.ExcelFile(name)

for i, sheet in enumerate(xlsx.sheet_names):    
    table = np.array(xlsx.parse(sheet))
    date_list = np.array([[all_dates[i][0]]]*len(table))    
    with_dates = np.append(table, date_list, axis=1)

    data_sheets.append(with_dates)
    names_to_find.append(sheet)

# find unique elements and return counts
unique, counts = np.unique(names_to_find,return_counts=True)

#find unique sheet names with frequency of 1
unique_set = unique[counts==1]

#find the indices where the unique sheet names exist 
idx_to_select = np.argwhere(np.isin(names_to_find, unique_set)).flatten()

#use list comprehension to subset data_sheets subset data_sheets
data_sheets = [data_sheets[i] for i in idx_to_select]

#convert to dataframe and rename columns
data_sheets = np.concatenate((data_sheets), axis=0)
df = pd.DataFrame(data_sheets, columns= [
    'stop_abbr', 'stop_name',
    'latitude', 'longitude', 
    'total_on', 'total_off', 'date_range'])
df.columns = [col.lower() for col in df]
df.columns = [col.replace(" ", "_") for col in df]

df.to_csv(exportname, index=False)